In [1]:
import importlib

In [2]:
import gc
import sys
import time
sys.path.append("/home/shaow/jupyter_base/pycharm_001")
base = "/home/shaow/jupyter_base/pycharm_001"
import torch
import experiment.model.dglSemanticModel as models
import graph_process as G
import obj.GraphBase as GB
import utils

In [3]:
appid_edge, appid_ros_edge, appid_app_edge, app_ros_edge, app_kw_edge, \
           id_index_map ,index_id_map, index_keyword_map, index_ros_map, index_app_map , \
           app_dict = G.process_year_graph(2019)



gb_2019 = GB.GraphBase(appid_edge, appid_ros_edge, appid_app_edge, app_ros_edge, app_kw_edge,
           id_index_map, index_id_map, index_ros_map, index_app_map, index_keyword_map ,
           app_dict)


8.0.19
10
socket 10.0.82.237:3306
done init 3564 nodes from apply id
244800
done init 593205 nodes from application


In [4]:
# importlib.reload(G)
gb = gb_2019.get_split_appid_map('F01')
t_data = G.get_graph_edges(gb)
i2w = gb.ikm

(2, 229)
<class 'int'>
<class 'numpy.int64'>
(2, 5652)
<class 'int'>
<class 'numpy.int64'>
(2, 5652)
<class 'int'>
<class 'numpy.int64'>
(2, 25724)
<class 'int'>
<class 'numpy.int64'>


In [5]:
train_data = t_data.train
valid_data = t_data.validation
test_data = t_data.test
# num_rels = t_data.num_rels
num_nodes = gb.node_count()
num_rels = gb.rel_count()

print(test_data.shape)
print(train_data.shape)
print(valid_data.shape)
test_graph, test_rel, test_norm = utils.build_test_graph(
        num_nodes, num_rels, train_data)
test_deg = test_graph.in_degrees(
            range(test_graph.number_of_nodes())).float().view(-1,1)
test_node_id = torch.arange(0, num_nodes, dtype=torch.long).view(-1, 1)
test_rel = torch.from_numpy(test_rel)
test_norm = models.node_norm_to_edge_norm(test_graph, torch.from_numpy(test_norm).view(-1, 1))

test_data = torch.from_numpy(test_data)
# use_cuda = torch.cuda.is_available()

(3680, 3)
(29835, 3)
(3742, 3)
Test graph:


/home/shaow/jupyter_base/pycharm_001/utils.py:130: RuntimeWarning: divide by zero encountered in true_divide
  norm = 1.0 / in_deg


In [6]:
importlib.reload(models)
model = models.LinkPredict(num_nodes, 256, num_rels, i2w,num_hidden_layers = 4)
model.train()
# print(model.is_leaf)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

True
init 4428 words vector
all word amount is 13896


In [7]:
model.eval()
print("start eval")
print(test_graph)
        # if use_cuda:
        #     test_graph = test_graph.to('cuda:0')
        #     test_node_id, test_rel, test_norm = test_node_id.cuda(), test_rel.cuda(), test_norm.cuda()
        # embed = model.embedding.embedding
        # embed = embed(test_node_id)
embed = model(test_graph, test_node_id, test_rel, test_norm)
        # torch.cuda.empty_cache()
        # gc.collect()
        # args.eval_batch_size = 500
mrr = utils.calc_mrr(embed, model.w_relation, test_data,
                             hits=[1, 3, 10], eval_bz=500)
print(mrr)

start eval
DGLGraph(num_nodes=20518, num_edges=59670,
         ndata_schemes={}
         edata_schemes={})
batch 0 / 8
batch 1 / 8
batch 2 / 8
batch 3 / 8
batch 4 / 8
batch 5 / 8
batch 6 / 8
batch 7 / 8
batch 0 / 8
batch 1 / 8
batch 2 / 8
batch 3 / 8
batch 4 / 8
batch 5 / 8
batch 6 / 8
batch 7 / 8
MRR (raw): 0.000172
Hits (raw) @ 1: 0.000000
Hits (raw) @ 3: 0.000000
Hits (raw) @ 10: 0.000000
0.00017181584553327411


In [56]:
models.semantic_base.wv.__contains__('微波光子信号产生')

False